In [1]:
# !pip uninstall -qy scikit-learn
# !pip install -q scikit-learn==1.5.2
# !pip install -qU xgboost

upload xgb_models.zip

In [2]:
!rm -rf xgb_model_*.json
!rm -rf xgb_model_*.pkl
!unzip -q xgb_models.zip
!unzip -q xgb_models_pkl.zip

unzip:  cannot find or open xgb_models.zip, xgb_models.zip.zip or xgb_models.zip.ZIP.


In [3]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

os.environ['WANDB_DISABLED'] = "true"

In [4]:
# !gdown 1qq5gDuEAqWnoviRBDYfbprLFD-MPQyNo
# !gdown 1mq9CWT9MXsH1Y-ihxVL4uEzufQjpBoer
!gdown 1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql

Downloading...
From: https://drive.google.com/uc?id=1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql
To: /content/processed_data.zip
100% 6.00M/6.00M [00:00<00:00, 17.9MB/s]


In [5]:
!rm -rf processed_data

!unzip -q processed_data.zip

In [6]:
!pip install -q transformers[torch] accelerate -U

!pip install -q datasets

!pip install -q transformers

!pip install -q sentence_transformers

!pip install -q auto-gptq optimum bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 33.1 MB/s eta 0:00:00
   ━━━━

In [7]:
import numpy as np

import pandas as pd

import os

from os import path

import glob

from tqdm.notebook import tqdm

from tqdm import tqdm

import string

import copy

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer,IntervalStrategy,BitsAndBytesConfig

from datasets import load_dataset, DatasetDict

# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split,KFold

from sklearn.metrics import classification_report,accuracy_score, f1_score

from sklearn.metrics import precision_recall_fscore_support

from sklearn.utils import shuffle

from sklearn.preprocessing import LabelEncoder

from scipy.special import expit

from peft import prepare_model_for_kbit_training

from peft import LoraConfig, get_peft_model,PeftModel


In [8]:
import torch

import torch.nn as nn

import torch.nn.functional as F

In [9]:
# import torch
# import numpy as np
# import random

# # Set random seed for Python's random module
# random.seed(42)

# # Set random seed for NumPy
# np.random.seed(42)

# # Set random seed for PyTorch
# torch.manual_seed(42)

# # If you're using GPU
# if torch.cuda.is_available():
#     torch.cuda.manual_seed(42)
#     torch.cuda.manual_seed_all(42)  # for multi-GPU setups
#     torch.backends.cudnn.deterministic = True
#     torch.backends.cudnn.benchmark = False

# # If you use Hugging Face's Transformers models
# from transformers import set_seed
# set_seed(42)


In [10]:


models_list_3=[
        # "FacebookAI/xlm-roberta-large",
      # "google-bert/bert-base-uncased",
        "infgrad/jasper_en_vision_language_v1",

#      "google-bert/bert-base-multilingual-uncased",

          ]



models_name_3=[
# "FacebookAI/xlm-roberta-large",
# "google-bert/bert-base-uncased",
        "infgrad/jasper_en_vision_language_v1",

# "bert",

]


In [11]:
kind="dev"

task="track_a"

# langs=["afr","amh","deu","eng","oro","ptbr","rus","som","sum","tir"]
# labels=['Anger','Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
# langs=["amh","arq","ary","chn","deu","esp","hau","hin","ibo","kin","mar","orm","pcm","ptbr","ptmz","ron","rus","som","sun","swa","swe","tat","tir","ukr","vmw","yor","ind","jav","xho","zul"]
langs=["eng"]
# langs=["afr"]


# processed_path=f"processed_data/{kinds[0]}/{tasks[0]}/{langs[3]}.csv"



# train_data=pd.read_csv(processed_path)


In [12]:
all_csv=pd.DataFrame()

for lang in langs:

    processed_path=f"processed_data/{kind}/{task}/{lang}.csv"
    # if kind=="dev":
    #    processed_path=processed_path.replace(lang,lang+task[-2:])
    if not os.path.isfile(processed_path):

      print("not found:",processed_path)

      continue
    train_data=pd.read_csv(processed_path)

    train_data.columns = train_data.columns.str.lower()

    all_csv = pd.concat([all_csv, train_data],ignore_index=True)




In [13]:
all_csv

,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,eng_dev_track_a_00001,NaN,NaN,NaN,NaN,NaN,eng,older sister 23 at the time is a scumbag stacy
1,eng_dev_track_a_00002,NaN,NaN,NaN,NaN,NaN,eng,and i laughed like this garhahagar because my ...
2,eng_dev_track_a_00003,NaN,NaN,NaN,NaN,NaN,eng,it overflowed and brown shitty diarrhea water ...
3,eng_dev_track_a_00004,NaN,NaN,NaN,NaN,NaN,eng,its very dark and foggy
4,eng_dev_track_a_00005,NaN,NaN,NaN,NaN,NaN,eng,then she tried to like have sex withstrangle e...
...,...,...,...,...,...,...,...,...
111,eng_dev_track_a_00112,NaN,NaN,NaN,NaN,NaN,eng,my heart was beating fast from excitement
112,eng_dev_track_a_00113,NaN,NaN,NaN,NaN,NaN,eng,a fraying rope stretches down from the rafters
113,eng_dev_track_a_00114,NaN,NaN,NaN,NaN,NaN,eng,so i cried my eyes out and did the drawing
114,eng_dev_track_a_00115,NaN,NaN,NaN,NaN,NaN,eng,never been so close to a group asswooping in m...


In [14]:
all_csv[all_csv["lang"]=="eng"]

,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,eng_dev_track_a_00001,NaN,NaN,NaN,NaN,NaN,eng,older sister 23 at the time is a scumbag stacy
1,eng_dev_track_a_00002,NaN,NaN,NaN,NaN,NaN,eng,and i laughed like this garhahagar because my ...
2,eng_dev_track_a_00003,NaN,NaN,NaN,NaN,NaN,eng,it overflowed and brown shitty diarrhea water ...
3,eng_dev_track_a_00004,NaN,NaN,NaN,NaN,NaN,eng,its very dark and foggy
4,eng_dev_track_a_00005,NaN,NaN,NaN,NaN,NaN,eng,then she tried to like have sex withstrangle e...
...,...,...,...,...,...,...,...,...
111,eng_dev_track_a_00112,NaN,NaN,NaN,NaN,NaN,eng,my heart was beating fast from excitement
112,eng_dev_track_a_00113,NaN,NaN,NaN,NaN,NaN,eng,a fraying rope stretches down from the rafters
113,eng_dev_track_a_00114,NaN,NaN,NaN,NaN,NaN,eng,so i cried my eyes out and did the drawing
114,eng_dev_track_a_00115,NaN,NaN,NaN,NaN,NaN,eng,never been so close to a group asswooping in m...


In [15]:
# float_columns = all_csv.select_dtypes(include=['float64']).columns
# # Check each column and convert only if there are no NaN values
# for col in float_columns:
#     if all_csv[col].isna().sum() == 0:  # No NaN values in the column
#         print(f"converting {kind} {task} {lang} column {col} to int")
#         all_csv[col] = all_csv[col].astype('int')
#     else:
#         print(f"converting {kind} {task} {lang} column {col} to int")

In [16]:
train_data=all_csv


In [17]:
# train_data = train_data.drop(columns=['disgust'],axis=1)


In [18]:

train_data=train_data.fillna(0)

train_data

,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,eng_dev_track_a_00001,0.0,0.0,0.0,0.0,0.0,eng,older sister 23 at the time is a scumbag stacy
1,eng_dev_track_a_00002,0.0,0.0,0.0,0.0,0.0,eng,and i laughed like this garhahagar because my ...
2,eng_dev_track_a_00003,0.0,0.0,0.0,0.0,0.0,eng,it overflowed and brown shitty diarrhea water ...
3,eng_dev_track_a_00004,0.0,0.0,0.0,0.0,0.0,eng,its very dark and foggy
4,eng_dev_track_a_00005,0.0,0.0,0.0,0.0,0.0,eng,then she tried to like have sex withstrangle e...
...,...,...,...,...,...,...,...,...
111,eng_dev_track_a_00112,0.0,0.0,0.0,0.0,0.0,eng,my heart was beating fast from excitement
112,eng_dev_track_a_00113,0.0,0.0,0.0,0.0,0.0,eng,a fraying rope stretches down from the rafters
113,eng_dev_track_a_00114,0.0,0.0,0.0,0.0,0.0,eng,so i cried my eyes out and did the drawing
114,eng_dev_track_a_00115,0.0,0.0,0.0,0.0,0.0,eng,never been so close to a group asswooping in m...


In [19]:
float_columns = train_data.select_dtypes(include=['float64']).columns

# Convert those columns to int
train_data[float_columns] = train_data[float_columns].astype('int')
train_data

,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,eng_dev_track_a_00001,0,0,0,0,0,eng,older sister 23 at the time is a scumbag stacy
1,eng_dev_track_a_00002,0,0,0,0,0,eng,and i laughed like this garhahagar because my ...
2,eng_dev_track_a_00003,0,0,0,0,0,eng,it overflowed and brown shitty diarrhea water ...
3,eng_dev_track_a_00004,0,0,0,0,0,eng,its very dark and foggy
4,eng_dev_track_a_00005,0,0,0,0,0,eng,then she tried to like have sex withstrangle e...
...,...,...,...,...,...,...,...,...
111,eng_dev_track_a_00112,0,0,0,0,0,eng,my heart was beating fast from excitement
112,eng_dev_track_a_00113,0,0,0,0,0,eng,a fraying rope stretches down from the rafters
113,eng_dev_track_a_00114,0,0,0,0,0,eng,so i cried my eyes out and did the drawing
114,eng_dev_track_a_00115,0,0,0,0,0,eng,never been so close to a group asswooping in m...


In [20]:
train_data.dtypes

,0
id,object
anger,int64
fear,int64
joy,int64
sadness,int64
surprise,int64
lang,object
clean_message,object


In [21]:
train_data = train_data[

train_data['clean_message'].isnull()==False]

train_data.reset_index()

,index,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,0,eng_dev_track_a_00001,0,0,0,0,0,eng,older sister 23 at the time is a scumbag stacy
1,1,eng_dev_track_a_00002,0,0,0,0,0,eng,and i laughed like this garhahagar because my ...
2,2,eng_dev_track_a_00003,0,0,0,0,0,eng,it overflowed and brown shitty diarrhea water ...
3,3,eng_dev_track_a_00004,0,0,0,0,0,eng,its very dark and foggy
4,4,eng_dev_track_a_00005,0,0,0,0,0,eng,then she tried to like have sex withstrangle e...
...,...,...,...,...,...,...,...,...,...
111,111,eng_dev_track_a_00112,0,0,0,0,0,eng,my heart was beating fast from excitement
112,112,eng_dev_track_a_00113,0,0,0,0,0,eng,a fraying rope stretches down from the rafters
113,113,eng_dev_track_a_00114,0,0,0,0,0,eng,so i cried my eyes out and did the drawing
114,114,eng_dev_track_a_00115,0,0,0,0,0,eng,never been so close to a group asswooping in m...


In [22]:
train_data

,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,eng_dev_track_a_00001,0,0,0,0,0,eng,older sister 23 at the time is a scumbag stacy
1,eng_dev_track_a_00002,0,0,0,0,0,eng,and i laughed like this garhahagar because my ...
2,eng_dev_track_a_00003,0,0,0,0,0,eng,it overflowed and brown shitty diarrhea water ...
3,eng_dev_track_a_00004,0,0,0,0,0,eng,its very dark and foggy
4,eng_dev_track_a_00005,0,0,0,0,0,eng,then she tried to like have sex withstrangle e...
...,...,...,...,...,...,...,...,...
111,eng_dev_track_a_00112,0,0,0,0,0,eng,my heart was beating fast from excitement
112,eng_dev_track_a_00113,0,0,0,0,0,eng,a fraying rope stretches down from the rafters
113,eng_dev_track_a_00114,0,0,0,0,0,eng,so i cried my eyes out and did the drawing
114,eng_dev_track_a_00115,0,0,0,0,0,eng,never been so close to a group asswooping in m...


In [23]:
label_columns =['anger', 'fear', 'joy', 'sadness', 'surprise']
# label_columns =['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']


In [24]:
train_data['lang'].value_counts()

,count
lang,
eng,116


In [25]:
sum(train_data['lang'].isna())

0

In [26]:
nan_rows = train_data[train_data.isna().any(axis=1)]
nan_rows
# Group by 'lang' and list the rows with NaNs for each language
# nan_by_language = nan_rows.groupby('lang').apply(lambda x: x)

# Display the result
# print(nan_by_language)

,id,anger,fear,joy,sadness,surprise,lang,clean_message


In [27]:
nan_rows[nan_rows["lang"]=="eng"]

,id,anger,fear,joy,sadness,surprise,lang,clean_message


In [28]:
train_data[train_data["lang"]=="deu"].dtypes

,0
id,object
anger,int64
fear,int64
joy,int64
sadness,int64
surprise,int64
lang,object
clean_message,object


In [29]:
nan_rows["lang"].value_counts()

,count
lang,


In [30]:
nan_rows.dtypes

,0
id,object
anger,int64
fear,int64
joy,int64
sadness,int64
surprise,int64
lang,object
clean_message,object


In [31]:
# Step 2: Count how many columns are not 0 or 1 for each language
def count_nonzero_nonone_rows(group):
    # Exclude the 'lang' column from counting if needed
    # Use boolean conditions to count values that are not 0 or 1
    return (group.drop(columns='anger') != 0) & (group.drop(columns='anger') != 1).sum(axis=1).sum()

# Group by 'lang' and apply the function to count non-zero, non-one values
non_zero_non_one_counts = train_data.groupby('lang').apply(count_nonzero_nonone_rows)

# Display the results
# result = pd.DataFrame({
#     'row_count': lang_row_counts,
#     'non_zero_non_one_count': non_zero_non_one_counts
# })

print(non_zero_non_one_counts)

             id   fear    joy  sadness  surprise   lang  clean_message
lang                                                                  
eng  0    False  False  False    False     False  False          False
     1    False  False  False    False     False  False          False
     2    False  False  False    False     False  False          False
     3    False  False  False    False     False  False          False
     4    False  False  False    False     False  False          False
...         ...    ...    ...      ...       ...    ...            ...
     111  False  False  False    False     False  False          False
     112  False  False  False    False     False  False          False
     113  False  False  False    False     False  False          False
     114  False  False  False    False     False  False          False
     115  False  False  False    False     False  False          False

[116 rows x 7 columns]


<ipython-input-31-06fbda4bf1b6>:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  non_zero_non_one_counts = train_data.groupby('lang').apply(count_nonzero_nonone_rows)


In [32]:
maxi=train_data['clean_message'].apply(len).max()
maxi

303

In [33]:
train_data[train_data['clean_message'].apply(len) == maxi]

,id,anger,fear,joy,sadness,surprise,lang,clean_message
39,eng_dev_track_a_00040,0,0,0,0,0,eng,swarms of people wandering around confused and...


In [34]:
train_data[train_data['clean_message'].apply(len) == maxi]["clean_message"].values

array(['swarms of people wandering around confused and unaware of where theyre going cutting people short as they wander into burger king and other such stores on the other side of the corridorcompletely unaware of the fact that they literally almost made me drop everything im carrying and fall flat on my face'],
      dtype=object)

In [35]:
# shuffle data
train_data=train_data.sample(frac=1)
# train_data=train_data.sample(100,random_state=42)


In [36]:
# train valid split
train,valid=train_test_split(train_data,test_size=0.2,random_state=42)

In [37]:
x_dev, y_dev = train_data['clean_message'].values.tolist(), train_data[label_columns].values.tolist()

In [38]:
y_dev_df = pd.DataFrame(y_dev, columns=label_columns)

In [39]:
device="cuda" if torch.cuda.is_available() else 'cpu'

In [40]:
MAX_LENGTH = 512

In [41]:
quantization_config = BitsAndBytesConfig(

    load_in_4bit=True,

    bnb_4bit_use_double_quant=True,

    bnb_4bit_quant_type="nf4",

    bnb_4bit_compute_dtype=torch.bfloat16,

)

In [42]:
models_list=models_list_3

models_name=models_name_3

x=models_list[0]

In [43]:
token=""

import huggingface_hub

if "gemma" in x or "Phi" in x:

  huggingface_hub.login(token=token)

In [44]:
USE_LORA=False

In [45]:
from transformers import AutoModel
from sentence_transformers import SentenceTransformer

In [46]:


torch.cuda.empty_cache()



print(x)
use_gpu = torch.cuda.is_available()
embedding_model = SentenceTransformer(
        x,
        trust_remote_code=True,
        device="cpu" if not use_gpu else "cuda",
        model_kwargs={
            "torch_dtype": torch.bfloat16 if use_gpu else torch.float32,
            "attn_implementation": "sdpa"
        },
        # vector_dim must be 12288, 1024, 512, 256
        ## 1024 is recommended
        # set is_text_encoder 'True', if you do not encode image
        config_kwargs={"is_text_encoder": True, "vector_dim": 1024},
    )
    # We can reduce the max_seq_length from the default of 2048 for faster encoding
embedding_model.max_seq_length = MAX_LENGTH

infgrad/jasper_en_vision_language_v1


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/246k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

In [47]:
def find_target_modules(model):
    # Initialize a Set to Store Unique Layers
    unique_layers = set()

    # Iterate Over All Named Modules in the Model
    for name, module in model.named_modules():
        # Check if the Module Type Contains 'Linear4bit'
        if "Linear4bit" in str(type(module)):
            # Extract the Type of the Layer
            layer_type = name.split('.')[-1]

            # Add the Layer Type to the Set of Unique Layers
            unique_layers.add(layer_type)

    # Return the Set of Unique Layers Converted to a List
    return list(unique_layers)
find_target_modules(embedding_model)

[]

In [48]:
# a=

In [49]:
embedding_model.eval() # embedding_model in training mode (dropout modules are activated)

if USE_LORA:

  # enable gradient check pointing

  embedding_model.gradient_checkpointing_enable()

  target_modules_dict={

      "microsoft/Phi-3.5-mini-instruct":'all-linear',
      "unsloth/Llama-3.2-1B-Instruct":["q_proj","k_proj","v_proj"],
      "unsloth/Qwen2.5-3B-Instruct-bnb-4bit":["q_proj","k_proj","v_proj"],
  }

  target_modules= target_modules_dict.get(x,["q_proj"])



  # enable quantized training

  embedding_model = prepare_model_for_kbit_training(embedding_model)

  config = LoraConfig(

    r=8,

    lora_alpha=32,

    # target_modules=["k_proj"],

    target_modules = target_modules,#TODOOOOOOOOOOOOO

    lora_dropout=0.1,

    bias="none",

    task_type="SEQ_CLS",

  )

  # LoRA trainable version of model

  embedding_model = get_peft_model(embedding_model, config)



  # trainable parameter count

  embedding_model.print_trainable_parameters()

device="cuda" if torch.cuda.is_available() else 'cpu'

In [50]:
device

'cuda'

In [51]:
embedding_model = embedding_model.to(device)

for param in embedding_model.parameters():
    param.data = param.data.contiguous()


In [52]:

# Custom Dataset for batching
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Tokenize a single example
        return self.texts[idx]


In [53]:
# Batch extraction function
def extract_embeddings(texts, model, batch_size=32):
    dataset = TextDataset(texts)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    embeddings = []
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Extracting embeddings"):
            # Move batch to device
            # input_ids = torch.Tensor(batch['input_ids']).squeeze().to(device)
            # attention_mask = torch.Tensor(batch['attention_mask']).squeeze().to(device)

            # Forward pass through the model
            cls_embeddings = model.encode(batch)
            embeddings.append(cls_embeddings)

    return np.vstack(embeddings)

In [54]:
# Extract embeddings for train and validation datasets
x_dev_embeddings = extract_embeddings(x_dev, embedding_model, batch_size=32)

Extracting embeddings: 100%|██████████| 4/4 [00:07<00:00,  1.82s/it]


In [55]:
# Flatten labels for XGBoost (multi-label classification)
y_dev_flat = np.array(y_dev)

In [56]:
import xgboost as xgb
import pickle

In [57]:
# Function to load XGBoost models
def load_xgb_models(label_columns):
    xgb_models = {}
    for label in label_columns:
        # model = xgb.XGBClassifier()
        # model.load_model(f"xgb_model_{label}.json")
        with open(f"xgb_model_{label}.pkl", 'rb') as f:
          model = pickle.load(f)
        xgb_models[label] = model
    return xgb_models

In [58]:
xgb_models = load_xgb_models(label_columns)

In [59]:

# Predict labels for the test data
test_predictions = {}
for label in label_columns:
    print(f"Predicting for label: {label}")
    test_predictions[label] = xgb_models[label].predict(x_dev_embeddings)

Predicting for label: anger
Predicting for label: fear
Predicting for label: joy
Predicting for label: sadness
Predicting for label: surprise


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [10:43:35] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [60]:
# Combine predictions into a DataFrame
predictions_df = pd.DataFrame(test_predictions)

In [61]:
train_data.reset_index(inplace=True)

In [115]:
# Add the predictions to the original test DataFrame
output_df = pd.concat([train_data[['id','lang']], predictions_df], axis=1)

In [129]:
# Create directory if it doesn't exist
output_dir = task
os.makedirs(output_dir, exist_ok=True)

# Iterate through the unique languages in the 'lang' column
for lang in output_df['lang'].unique():
    # Filter rows for the current language
    lang_df = output_df[output_df['lang'] == lang]
    lang_df.drop(columns=['lang'],inplace=True)

    # Define the output file path for the current language
    lang_csv_file = os.path.join(output_dir, f"pred_{lang}.csv")

    # Sort the DataFrame by the 'id' column
    lang_df.sort_values(by='id', inplace=True)

    # Save the filtered DataFrame to a CSV file
    lang_df.to_csv(lang_csv_file, index=False)

    # Print a confirmation message
    print(f"Predictions for language '{lang}' saved to {lang_csv_file}")

Predictions for language 'eng' saved to track_a/pred_eng.csv


In [130]:
!zip -r {task}.zip {task}

updating: track_a/ (stored 0%)
updating: track_a/pred_eng.csv (deflated 88%)


download track_a|b|c.zip